In [1]:
from bancor_research import DEFAULT
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp as sBancorDapp
from bancor_research.bancor_emulator.v3.spec.network import BancorDapp as eBancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {tkn_name : DEFAULT.WHITELIST[tkn_name] for tkn_name in DEFAULT.WHITELIST if tkn_name in ['eth', 'link', 'wbtc']}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
dapps = [BancorDapp(whitelisted_tokens=whitelisted_tokens) for BancorDapp in [sBancorDapp, eBancorDapp]]

def assertAndDisplay(decimals = -1):
    frames = [v3.describe(decimals) for v3 in dapps]
    diff = frames[0].compare(frames[1])
    assert diff.empty, diff
    return frames[0]

for v3 in dapps:
    v3.create_user('Trader')

    v3.set_user_balance(tkn_amt='102', tkn_name='eth', user_name='Alice')
    v3.set_user_balance(tkn_amt='102', tkn_name='wbtc', user_name='Bob')
    v3.set_user_balance(tkn_amt='10002', tkn_name='link', user_name='Charlie')
    v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')
    v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')

    v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')
    v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

    for tkn_name in whitelisted_tokens:
        v3.enable_trading(tkn_name)

    v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)
    v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)


In [2]:
assertAndDisplay(decimals=6)

bnt           eth          link  \
1 Account Alice                         0.000000      1.000000      0.000000   
          Bob                           0.000000      0.000000      0.000000   
          Charlie                       0.000000      0.000000      1.000000   
          Trader                     1977.615297      0.000000      0.001900   
2 Pool    a: TKN Staked Balance                0    101.000000  10003.424242   
          b: TKN Trading Liquidity             0     20.000000   3336.331433   
          c: BNT Trading Liquidity             0  20000.000000  20013.993917   
          d: BNT Current Funding               0  20000.000000  20015.980730   
          e: Spot Rate                         0   1000.000000      5.998803   
          f: Average Rate                      0   1000.000000      6.000000   
          g: Average Inverse Rate              0      0.001000      0.166667   
3 Network Master Vault              60022.384703    101.000000  10003.998100   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                      0             0             0   
          Rewards Vault                        0             0             0   

                                            wbtc         bnbnt       bneth  \
1 Account Alice                         0.000000      0.000000  101.000000   
          Bob                           1.000000      0.000000    0.000000   
          Charlie                       0.000000      0.000000    0.000000   
          Trader                        0.000000      0.000000    0.000000   
2 Pool    a: TKN Staked Balance       101.000000             0           0   
          b: TKN Trading Liquidity      1.250000             0           0   
          c: BNT Trading Liquidity  20000.000000             0           0   
          d: BNT Current Funding    20000.000000             0           0   
          e: Spot Rate              16000.000000             0           0   
          f: Average Rate           16000.000000             0           0   
          g: Average Inverse Rate       0.000062             0           0   
3 Network Master Vault                101.000000             0           0   
          Protection Vault              0.000000             0           0   
          Protocol Equity                      0  60000.000000    0.000000   
          Rewards Vault                        0      0.000000    0.000000   

                                          bnlink      bnwbtc      vbnt  
1 Account Alice                         0.000000    0.000000  0.000000  
          Bob                           0.000000  101.000000  0.000000  
          Charlie                   10001.000000    0.000000  0.000000  
          Trader                        0.000000    0.000000  0.000000  
2 Pool    a: TKN Staked Balance                0           0         0  
          b: TKN Trading Liquidity             0           0         0  
          c: BNT Trading Liquidity             0           0         0  
          d: BNT Current Funding               0           0         0  
          e: Spot Rate                         0           0         0  
          f: Average Rate                      0           0         0  
          g: Average Inverse Rate              0           0         0  
3 Network Master Vault                         0           0         0  
          Protection Vault                     0           0         0  
          Protocol Equity               0.000000    0.000000         0  
          Rewards Vault                 0.000000    0.000000         0

# Staking TKN

The ema serves as a detection mechanism for unusual spikes in the price of an asset against BNT, and prevents changes to the depth of the trading pools to protect the system from harm. To illustrate the effects of the ema, the current state of the system in the present narrative will be carried forward, with each of Alice, Bob, and Charlie providing one additional TKN to the system. Unlike the prior examples where liquidity is added, both the ema, and the pool token valuation need to be taken under consideration.

Before the demonstration continues, recall that each of the pools have a preset 4,000 BNT funding limit at this point in the narrative, and up to this point in time all three have only exhausted half of that amount. Further, recall that the ability to increase the available trading liquidity is determined by the vault balance of the TKN in question, and the current depth of the pool, rather than the amount being contributed by a user during any deposit event.

In [3]:
timestamp = 0
for v3 in dapps:
    v3.deposit(tkn_amt='1', tkn_name='eth',  user_name='Alice', timestamp=timestamp)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      1.0000   
          Trader                     1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance              0    102.0000  10003.4242   
          b: TKN Trading Liquidity           0     40.0000   3336.3314   
          c: BNT Trading Liquidity           0  40000.0000  20013.9939   
          d: BNT Current Funding             0  40000.0000  20015.9807   
          e: Spot Rate                       0   1000.0000      5.9988   
          f: Average Rate                    0   1000.0000      6.0000   
          g: Average Inverse Rate            0      0.0010      0.1667   
3 Network Master Vault              80022.3847    102.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  102.0000   
          Bob                           1.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0000      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0000           0         0   
          b: TKN Trading Liquidity      1.2500           0         0   
          c: BNT Trading Liquidity  20000.0000           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              16000.0000           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network Master Vault                101.0000           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  79994.6745    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

Alice now deposits 1 ETH. Since no fees have yet accrued to the bnETH pool tokens (i.e. ETH has not yet been the target of any particular trade), the value of the bnETH pool token remains unchanged. Her 1 ETH is added to the vault, the staking ledger is updated, and a new pool token is created for Alice.

Assume that this deposit is occurring on a block wherein no actions have taken place on the ETH pool. Therefore, the ema has not been updated in this block, and it remains where it was. This allows for the effective price quoted by the pool to be evaluated with a view to detecting artificial price manipulation. The protocol's confidence is determined by the relative deviation between the ema and spot prices; the protocol will only allow the pool's trading liquidity to be changed if the difference between these values is small.

In this instance, the addition of Alice's 1 ETH occurs when the ema on ETH remains at 1,000, whereas its spot rate is 907.26. The deviation between the spot rate and the ema is measured relative to the ema, as follows:

Since Δ is fixed at 0.01, the equation above can be expressed as the following boolean:

![](https://lh5.googleusercontent.com/blsejE9xJDrmvWZPTX9pyIZnAHe6_FiCmwoKm-IryJ0w678oabuREMMxDo5PrZRbSg68QLAFxG1COySwZTquiD0rfdRxK63u4yGw8In-kcsQFYFuwJWP4JKXebJloN_mkTZ5deQ7YCEaTO5esA)\
where r is the spot rate. In this case, Δ is 0.0927; the tolerance of the protocol is 0.01. Therefore, the protocol will not allow the trading liquidity to be changed on this block. However, Alice's 1 ETH is still accepted into the vault, and she is issued bnETH pool tokens as normal. Since there is no change to the trading liquidity, the protocol will not mint BNT, and will not issue itself bnBNT.

Bob now deposits 1 wBTC. In contrast to Alice’s situation, fees have accumulated on the wBTC pool token. Therefore, the rate of pool token issuance is affected. The staking ledger is reporting a staked amount of wBTC of 101.00045, and the supply of bnwBTC is 101. Therefore, the rate of bnwBTC to wBTC is 0.999995543, and this quotient determines the number of bnwBTC pool tokens Bob will receive.

Assume that this deposit is occurring on the same block as Alice’s deposit, where the trade between ETH and wBTC had been executed on the prior block. Therefore, the wBTC ema has not been updated in this block, and also remains at the genesis rate.

In this instance, the addition of Bob’s 1 wBTC coccurs when the ema on wBTC remains at 16,000, whereas its spot rate is at 17,534. The deviation (Δ) between the spot rate and the ema is 0.0959, and as before, the protocol will not allow the trading liquidity to be changed on this block. Regardless, the vault still accepts Bob’s wBTC, and issues him bnwBTC as normal.


In [4]:
timestamp = 0
for v3 in dapps:
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob', timestamp=timestamp)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                          0.0000      0.0000      0.0000   
          Bob                            0.0000      0.0000      0.0000   
          Charlie                        0.0000      0.0000      1.0000   
          Trader                      1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance               0    102.0000  10003.4242   
          b: TKN Trading Liquidity            0     40.0000   3336.3314   
          c: BNT Trading Liquidity            0  40000.0000  20013.9939   
          d: BNT Current Funding              0  40000.0000  20015.9807   
          e: Spot Rate                        0   1000.0000      5.9988   
          f: Average Rate                     0   1000.0000      6.0000   
          g: Average Inverse Rate             0      0.0010      0.1667   
3 Network Master Vault              100022.3847    102.0000  10003.9981   
          Protection Vault               0.0000      0.0000      0.0000   
          Protocol Equity                     0           0           0   
          Rewards Vault                       0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  102.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0000      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       102.0000           0         0   
          b: TKN Trading Liquidity      2.5000           0         0   
          c: BNT Trading Liquidity  40000.0000           0         0   
          d: BNT Current Funding    40000.0000           0         0   
          e: Spot Rate              16000.0000           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network Master Vault                102.0000           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  99989.3490    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  102.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

Charlie now deposits 1 LINK. As was the case for Bob, the fee accrual on bnLINK will affect the issuance of pool tokens. The staking ledger is reporting a staked amount of LINK of 10,003.42, and the supply of bnLINK is 10,001. Therefore, the rate of bnLINK to LINK is 0.999758, and the issuance of bnLINK to Charlie is determined by this number.

Assume that this deposit occurs on the block following the two LINK trades previously discussed. In this instance, the addition of Charlie’s 1 LINK occurs when the ema on LINK is 5.9998, whereas its spot rate is at 5.9988. The deviation (Δ) between the spot rate and the ema is 0.00017, which is inside the tolerance levels of the protocol, and the protocol will allow the trading liquidity to update.

Trading liquidity always updates according to a strict set of criteria. In simple terms, the protocol will always try to increase the liquidity by as much as possible, up to a maximum BNT liquidity increase of 2× relative to the current depth, and without exceeding the preset funding limit. In this example, the current BNT depth on the LINK pool is 20,013.993668. Therefore, with a sufficient funding limit, the pool could double in size, up to a maximum BNT depth of 40,027.98734 BNT. However, the protocol has previously contributed 20,000 BNT of its 40,000 BNT funding limit - it can only increase the trading liquidity by a maximum of 20,000 BNT. The protocol will always default to the smallest of these two values.

By convention, the protocol trusts the ema rate over the spot price. Therefore, in this example, the BNT/LINK quotient is treated as 5.9998. The maximum allowed increase is the BNT funding limit, 20,000 BNT. Therefore, 3333.998732 LINK and 20,000 BNT are added to the trading liquidity. As the protocol is minting BNT, it is simultaneously issuing new bnBNT pool tokens for itself. Since the bnBNT pool token has appreciated in value, this will need to be taken into account.

In [5]:
timestamp = 1
for v3 in dapps:
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie', timestamp=timestamp)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                          0.0000      0.0000      0.0000   
          Bob                            0.0000      0.0000      0.0000   
          Charlie                        0.0000      0.0000      0.0000   
          Trader                      1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance               0    102.0000  10004.4242   
          b: TKN Trading Liquidity            0     40.0000   6672.1302   
          c: BNT Trading Liquidity            0  40000.0000  40027.9878   
          d: BNT Current Funding              0  40000.0000  40029.9746   
          e: Spot Rate                        0   1000.0000      5.9993   
          f: Average Rate                     0   1000.0000      5.9999   
          g: Average Inverse Rate             0      0.0010      0.1667   
3 Network Master Vault              120036.3786    102.0000  10004.9981   
          Protection Vault               0.0000      0.0000      0.0000   
          Protocol Equity                     0           0           0   
          Rewards Vault                       0           0           0   

                                          wbtc        bnbnt     bneth  \
1 Account Alice                         0.0000       0.0000  102.0000   
          Bob                           0.0000       0.0000    0.0000   
          Charlie                       0.0000       0.0000    0.0000   
          Trader                        0.0000       0.0000    0.0000   
2 Pool    a: TKN Staked Balance       102.0000            0         0   
          b: TKN Trading Liquidity      2.5000            0         0   
          c: BNT Trading Liquidity  40000.0000            0         0   
          d: BNT Current Funding    40000.0000            0         0   
          e: Spot Rate              16000.0000            0         0   
          f: Average Rate           16000.0000            0         0   
          g: Average Inverse Rate       0.0001            0         0   
3 Network Master Vault                102.0000            0         0   
          Protection Vault              0.0000            0         0   
          Protocol Equity                    0  119998.0137    0.0000   
          Rewards Vault                      0       0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  102.0000  0.0000  
          Charlie                   10001.9998    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

The staking ledger is reporting a staked amount of BNT of 60,022.384, and the supply of bnBNT is 60,000. Therefore, the rate of bnBNT to BNT is 0.999627065. As the protocol mints 20,000 BNT to increase the trading liquidity, this amount is added to the BNT staking ledger, and the protocol issues itself 19,992.5413 bnBNT pool tokens.